## 載入相關套件

In [1]:
%matplotlib inline

In [2]:
import vectorbt as vbt

In [3]:
from numba import njit

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 從Yahoo finance下載股價資料，載入過去五年每日元大高股息ETF(0056)收盤資料

In [5]:
data = pd.read_csv(r'C:\Users\npc94\OneDrive\桌面\0056.TW.csv',index_col=0,parse_dates=True)

price= data['Close']

In [6]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-17,23.370001,23.49,23.370001,23.440001,23.440001,712583
2017-01-18,23.480000,23.48,23.410000,23.420000,23.420000,371489
2017-01-19,23.400000,23.40,23.350000,23.389999,23.389999,300555
2017-01-20,23.400000,23.43,23.350000,23.350000,23.350000,603250
2017-01-23,23.350000,23.49,23.350000,23.480000,23.480000,544355


In [7]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-10,33.250000,33.299999,33.119999,33.270000,33.270000,9915525
2022-01-11,33.290001,33.290001,33.160000,33.200001,33.200001,13666824
2022-01-12,33.310001,33.380001,33.049999,33.099998,33.099998,21269312
2022-01-13,33.099998,33.279999,33.099998,33.200001,33.200001,10983305
2022-01-14,33.220001,33.220001,32.860001,33.150002,33.150002,27153827


## 設定資料頻率為"日"，並設定交易只看前一次的報酬，前一次報酬為正就買進股票

In [8]:
vbt.settings.array_wrapper['freq'] = 'd'
risk_free=0.01

entries = np.log(price/price.shift(1))  > 0
exits = price.apply(np.log).diff(1)  < 0
               
portfolio = vbt.Portfolio.from_signals(price, entries, exits, fees=0,init_cash=10000)

## 交易進出資料

In [9]:
portfolio.orders.records_readable

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,Close,2017-01-23,425.894378,23.480000,0.0,Buy
1,1,Close,2017-02-16,425.894378,24.480000,0.0,Sell
2,2,Close,2017-02-18,424.334326,24.570000,0.0,Buy
3,3,Close,2017-02-20,424.334326,24.370001,0.0,Sell
4,4,Close,2017-02-22,424.160310,24.379999,0.0,Buy
...,...,...,...,...,...,...,...
545,545,Close,2022-01-06,451.913551,33.410000,0.0,Sell
546,546,Close,2022-01-10,453.815201,33.270000,0.0,Buy
547,547,Close,2022-01-11,453.815201,33.200001,0.0,Sell
548,548,Close,2022-01-13,453.815201,33.200001,0.0,Buy


## 交易圖表繪製

In [10]:
fig = price.vbt.plot(trace_kwargs=dict(name='Close'))
price.shift(1).vbt.plot(trace_kwargs=dict(name='momentum strategy'), fig=fig)
portfolio.positions.plot(close_trace_kwargs=dict(visible=False), fig=fig)

FigureWidget({
    'data': [{'name': 'Close',
              'showlegend': True,
              'type': 'scatter…

## 交易獲利統計

In [11]:
portfolio.total_profit()

5043.974834872017

## 交易數據圖表統計

In [12]:
portfolio.stats()

Start                               2017-01-17 00:00:00
End                                 2022-01-14 00:00:00
Period                               1219 days 00:00:00
Start Value                                     10000.0
End Value                                  15043.974835
Total Return [%]                              50.439748
Benchmark Return [%]                          41.424917
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              16.507809
Max Drawdown Duration                 186 days 00:00:00
Total Trades                                        275
Total Closed Trades                                 275
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  43.272727
Best Trade [%]                                 4.736627
Worst Trade [%]                               -4